In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd 
from datetime import date,datetime,timedelta
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

In [2]:
companies_list = pd.read_csv("company_name.csv")
companies_list.head()

,Company Names
0,Apple Inc
1,Microsoft Corporation
2,Alphabet Inc
3,"Amazoncom, Inc"
4,Berkshire Hathaway Inc


In [3]:
len(companies_list)

4231

In [4]:
date_range = [(date.fromisoformat("2002-01-01"),date.fromisoformat("2003-01-01")),
              (date.fromisoformat("2003-01-01"),date.fromisoformat("2006-01-01")),
              (date.fromisoformat("2006-01-01"),date.fromisoformat("2009-01-01")),
              (date.fromisoformat("2009-01-01"),date.fromisoformat("2012-01-01")),
              (date.fromisoformat("2012-01-01"),date.fromisoformat("2015-01-01")),
              (date.fromisoformat("2015-01-01"),date.fromisoformat("2018-01-01")),
              (date.fromisoformat("2018-01-01"),date.fromisoformat("2021-01-01")),
              (date.fromisoformat("2021-01-01"),date.today())]
search_form_keys = r' and "Event Brief" and ns=ntra and rst=fndw '
search_form_ID = r'ftx'
date_select_ID = r'dr'
date_select_text = r'Enter date range...'
date_from_month_ID = r'frm'
date_from_day_ID = r'frd'
date_from_year_ID = r'fry'
date_to_month_ID = r'tom'
date_to_day_ID = r'tod'
date_to_year_ID = r'toy'
company_arrow_XPATH = r'//*[@id="coTab"]/div[1]'
company_promote_XPATH = r'//*[@id="coMnu"]/ul/li[1]/a[3]'
company_text_ID = r'coTxt'
search_button_ID = r'btnSBSearch'
select_all_files_button_XPATH =r'//*[@id="selectAll"]/input'
pdf_button_XPATH = r'//*[@id="listMenuRoot"]/li[6]/a'
headline_button_XPATH = r'//*[@id="listMenu-id-4"]/li[3]/a'

In [5]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : r'C:\Users\yomaa\Documents\Github\University-of-Chicago\Capstone I\temp',"download.prompt_for_download": False,"disable-features":"DownloadUI"}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(options=chrome_options)
driver.get("http://proxy.uchicago.edu/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S003Wv90Gb95DEs5DEnODYqMDEsND7yMHn0VrJoWXNG2TBX4cjBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQQAA")

In [6]:
downloads_tracker = {
    'company' : [],
    'from_date' : [],
    'to_date' : [],
    'status' : [],
    'reason' : []
}

In [8]:
for i in range(100,201,1):
    storage_path = r'C:\\Users\\yomaa\\Documents\\Github\\University-of-Chicago\\Capstone I\\Files\\' + companies_list.iloc[i,0]
    if not os.path.exists(storage_path):
        os.mkdir(storage_path)
    for date in date_range:
        status = False
        reason = ""
        driver.get("http://proxy.uchicago.edu/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S003Wv90Gb95DEs5DEnODYqMDEsND7yMHn0VrJoWXNG2TBX4cjBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQQAA")
        from_date = date[0]
        to_date = date[1]
        company = companies_list.iloc[i,0]
        try:
        # Selecting the Date Selector and setting it to explicitly select dates 
            date_select = Select(driver.find_element(By.ID,date_select_ID))
            date_select.select_by_visible_text(date_select_text)

            # Setting the from date
            from_date_month = driver.find_element(By.ID,date_from_month_ID)
            from_date_month.send_keys(str(from_date.month))
            from_date_day = driver.find_element(By.ID,date_from_day_ID)
            from_date_day.send_keys(str(from_date.day))
            from_date_year = driver.find_element(By.ID,date_from_year_ID)
            from_date_year.send_keys(str(from_date.year))

            # Setting the To date
            to_date_month = driver.find_element(By.ID,date_to_month_ID)
            to_date_month.send_keys(str(to_date.month))
            to_date_day = driver.find_element(By.ID,date_to_day_ID)
            to_date_day.send_keys(str(to_date.day))
            to_date_year = driver.find_element(By.ID,date_to_year_ID)
            to_date_year.send_keys(str(to_date.year))

            # Clicking the company button
            company_button = driver.find_element(By.XPATH,company_arrow_XPATH)
            company_button.click()

            # Selecting the company
            company_form = driver.find_element(By.ID,company_text_ID)
            search_form = driver.find_element(By.ID,search_form_ID)
            company_form.send_keys(company,Keys.ENTER,Keys.ENTER)
            search_form.click()
            time.sleep(2)
            company_promote_test = driver.find_elements(By.XPATH,company_promote_XPATH)
            if company_promote_test == []:
                company_form.send_keys(Keys.ENTER,Keys.ENTER)
            time.sleep(2)
            company_promote = driver.find_element(By.XPATH,company_promote_XPATH)
            company_promote.click()
            
            # Setting the Search form textbox 
            search_form = driver.find_element(By.ID,search_form_ID)
            search_form.send_keys(search_form_keys)

            # Clicking the search button
            search_button = driver.find_element(By.ID,search_button_ID)
            search_button.click()
            #Selecting all files
            time.sleep(2)
            select_all_files_checkbox_search = driver.find_elements(By.XPATH,select_all_files_button_XPATH)
            if select_all_files_checkbox_search == []:
                status = False
                reason = "No Files"
            else:
                select_all_files_checkbox = driver.find_element(By.XPATH,select_all_files_button_XPATH)
                select_all_files_checkbox.click()
                time.sleep(2)
                # Downloading All  Files
                from_date
                new_file_name = companies_list.iloc[i,0] + "_" + from_date.strftime("%Y%m%d") + "_" + to_date.strftime("%Y%m%d") + ".pdf"
                driver.execute_script(r"viewProcessing('../pps/default.aspx?pp=PDF&ppstype=HeadlineWithArticleTOC',true);resetAction(); if(window.event)window.event.cancelBubble = true;return false;")
                time.sleep(5)
                for filename in os.listdir(prefs['download.default_directory']):
                    f = os.path.join(prefs['download.default_directory'], filename)
                    # checking if it is a file
                    if os.path.isfile(f):
                        os.replace(f, storage_path + "\\" + new_file_name)
                status = True
                reason = ""
        except Exception as e:
            status = False
            reason = str(e)
        downloads_tracker['company'].append(companies_list.iloc[i,0])
        downloads_tracker['from_date'].append(from_date)
        downloads_tracker['to_date'].append(to_date)
        downloads_tracker['status'].append(status)
        downloads_tracker['reason'].append(reason)

In [9]:
output_result = pd.DataFrame(downloads_tracker)
output_result

,company,from_date,to_date,status,reason
0,"Moderna, Inc",2002-01-01,2003-01-01,False,No Files
1,"Moderna, Inc",2003-01-01,2006-01-01,False,No Files
2,"Moderna, Inc",2006-01-01,2009-01-01,False,No Files
3,"Moderna, Inc",2009-01-01,2012-01-01,False,No Files
4,"Moderna, Inc",2012-01-01,2015-01-01,False,No Files
...,...,...,...,...,...
803,"Ulta Beauty, Inc",2009-01-01,2012-01-01,False,No Files
804,"Ulta Beauty, Inc",2012-01-01,2015-01-01,False,No Files
805,"Ulta Beauty, Inc",2015-01-01,2018-01-01,False,No Files
806,"Ulta Beauty, Inc",2018-01-01,2021-01-01,True,


In [10]:
output_result.to_csv("Run2.csv")

In [14]:
import shutil

company_names_list = output_result['company'].unique.to_list()
output_folder = r'C:\\Users\\yomaa\\Documents\Github\\University-of-Chicago\\Capstone I\\Run2 Files\\'
input_folder = r'C:\\Users\\yomaa\\Documents\\Github\\University-of-Chicago\\Capstone I\\Files\\'
processed = []
for i in os.listdir(input_folder):
    if i in company_names_list:
        original = input_folder + i
        target = output_folder + i
        shutil.move(original, target)
        processed.append(i)